# Correlation Matrix with Pytorch

In [1]:
%load_ext autoreload

In [2]:
import os
import sys
print(os.listdir('../'))

['teste', 'pyproject.toml', 'report', 'weights', 'notebooks', 'requirements.txt', 'data', '.gitignore', 'README.md', 'Makefile', 'parkinson', '.git']


In [3]:
import sys
# sys.path.append('/home/labic/merlin_codes/dl/Parkinson-Diagnosis-Deeplearning')
sys.path.append('../')

import os
import time
    
import parkinson

import torch
from torch import nn

import pandas as pd
import networkx as nx
import numpy as np

import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

SyntaxError: invalid syntax (graph.py, line 130)

In [ ]:
RDN = 50
N_CLASSES = 2
BATCH_SIZE = 32
N_EPOCHS = 200
PATIENCE = 20
LR = 0.0001

## Data Processing
### Reading

In [ ]:
parkinson_data = parkinson.utils.data.batch_read('../data/PDs_columns')
control_data = parkinson.utils.data.batch_read('../data/Controls_columns')

control_atlas_data = parkinson.utils.data.select_atlas_columns(control_data, 'AAL3')
parkinson_atlas_data = parkinson.utils.data.select_atlas_columns(parkinson_data, 'AAL3')

100%|██████████| 66/66 [00:01<00:00, 38.27it/s]


In [ ]:
correlation_method = "dtw"  # ou "pearson"

In [ ]:
parkinson_correlation_matrix = parkinson.utils.graph.compute_correlation_matrix(
    parkinson_atlas_data, method=correlation_method, group='parkinson'
)

Total: 153 pacientes
Já computados: 0
Restantes: 153

Processando batch 0 a 5...


KeyboardInterrupt: 

In [ ]:
control_correlation_matrix = parkinson.utils.graph.compute_correlation_matrix(
    control_atlas_data, method=correlation_method, group='control'
)

Carregando cache existente de cache_dtw_control.pkl...
Total: 66 pacientes
Já computados: 66
Restantes: 0

Resultado final salvo em cache_dtw_control_final.pkl


In [ ]:
# com pearson (descomentar se quiser usar pearson ao invés de DTW)
# parkinson_correlation_matrix = [parkinson.utils.graph.compute_correlation_matrix(time_series) for time_series in parkinson_atlas_data]
# control_correlation_matrix = [parkinson.utils.graph.compute_correlation_matrix(time_series) for time_series in control_atlas_data]

In [ ]:
X = parkinson.utils.data.concatenate_data(parkinson_correlation_matrix, control_correlation_matrix)
y = parkinson.utils.data.concatenate_data([1 for _ in range(len(parkinson_data))], [0 for _ in range(len(control_data))])

X, y = parkinson.utils.data.filter_data(X, y)

### Split  
- 60% treino
- 20% validação
- 20% teste

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=RDN, stratify=y, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=RDN, stratify=y_trainval, shuffle=True)

ros = RandomOverSampler(random_state=RDN)
X_train, y_train = ros.fit_resample(X_train, y_train)

### Dataloaders

In [ ]:
train_loader = parkinson.utils.data.get_torch_dataloader(X_train, y_train, batch_size=BATCH_SIZE)
val_loader = parkinson.utils.data.get_torch_dataloader(X_val, y_val, batch_size=BATCH_SIZE)
test_loader = parkinson.utils.data.get_torch_dataloader(X_test, y_test, batch_size=BATCH_SIZE)

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = parkinson.NetworkModels.SimpleMLP(input_dim=X_train.shape[1], hidden_dim=16, output_dim=2)
class_weights = parkinson.utils.data.get_torch_class_weights(y_train)

In [ ]:
model.train()
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [ ]:
%%time
out = parkinson.utils.train.train(model, train_loader, val_loader, class_weights, device, N_EPOCHS,  PATIENCE,LR)

train-loss: 0.7036  train-acc: 0.5000 | val-loss: 0.7667  val-acc: 0.2955:  16%|█▋        | 33/200 [00:11<00:59,  2.79it/s]

Early stopping at epoch 34
CPU times: user 7.86 s, sys: 9.67 s, total: 17.5 s
Wall time: 12.6 s


In [ ]:
metrics = parkinson.utils.train.evaluate(model, test_loader, device)
print('Metrics:', metrics)
print('Done.')

Preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Metrics: {'acc': 0.29545454545454547, 'f1': 0.13476874003189793, 'recall': 0.29545454545454547, 'precision': 0.08729338842975208}
Done.


/home/calixto/miniconda3/envs/trabRN/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import matplotlib.pyplot as plt

val_loss =  s['val_loss']
plt.plot(np.arange(len(val_loss)), val_loss)

NameError: name 's' is not defined